# Environment Setup

## Import Libraries

In [21]:
# Import required libraries

from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Read Data

In [22]:
# Read previously processed test & train data

train = pd.read_pickle('../../data/processed/knn-model/train.pkl')
test = pd.read_pickle('../../data/processed/knn-model/test.pkl')

In [23]:
# View loaded train data

train.head()

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
33780,794029,139518,2016-08-20,4,I'm not sure why several folks had trouble wit...,magnolia bakery s vanilla birthday cake and fr...,50,24386,2005-09-30,"['60-minutes-or-less', 'time-to-make', 'course...","[645.4, 38.0, 335.0, 7.0, 9.0, 77.0, 34.0]",22,"['preheat oven to 350', 'grease and lightly fl...",these comments are from their cookbook: \r\n\r...,"['unsalted butter', 'sugar', 'eggs', 'self ris...",8.0
44565,128473,536070,2018-10-03,5,"Delicious scones Bonnie, absolutely delicious....",strawberry shortcake scones,38,400708,2018-06-25,"['60-minutes-or-less', 'time-to-make', 'course...","[256.8, 10.0, 123.0, 4.0, 4.0, 20.0, 15.0]",20,"['preheat oven to 425f', 'line baking sheet wi...",with strawberries filling my garden found this...,"['all-purpose flour', 'granulated sugar', 'sal...",8.0
56295,2001345166,20376,2017-02-01,5,I am told that these are the best cookies I've...,nestle oatmeal scotchies,25,28846,2002-02-22,"['30-minutes-or-less', 'time-to-make', 'course...","[1311.5, 109.0, 520.0, 47.0, 18.0, 244.0, 53.0]",14,"['preheat oven to 375 degrees f', 'combine flo...",one of our families favorite cookies. i often ...,"['unsifted flour', 'baking soda', 'salt', 'cin...",11.0
61487,2001276950,293275,2017-02-11,5,After trying several different stew recipes I ...,quick and easy beef stew,25,742802,2008-03-21,"['30-minutes-or-less', 'time-to-make', 'course...","[496.2, 21.0, 97.0, 871.0, 48.0, 28.0, 23.0]",8,['in a medium mixing bowl put flour and teaspo...,"we don't eat red meat very often in our house,...","['sirloin tip roast', 'potatoes', 'carrots', '...",11.0
16305,1741243,257554,2016-01-11,2,"This was pretty bland, though I made it pretty...",crock pot beef or lamb casserole,270,329769,2007-10-08,"['course', 'main-ingredient', 'cuisine', 'prep...","[1583.1, 228.0, 28.0, 18.0, 46.0, 307.0, 12.0]",13,['place the diced meat and flour in a plastic ...,an economical meal for a family. crock pot/slo...,"['potatoes', 'celery ribs', 'carrots', 'onions...",14.0


In [24]:
# View Loaded test data

test.head()

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
12224,2000297339,177369,2018-02-18,0,I made this today....I had two one-pound piece...,a 1 pot roast chuck steak,135,311003,2006-07-13,"['time-to-make', 'course', 'main-ingredient', ...","[74.5, 5.0, 40.0, 3.0, 0.0, 2.0, 3.0]",11,"['in 12-inch dutch oven , over medium-high hea...",a great recipe for a chuck underblade steak th...,"['salad oil', 'underblade steak', 'steak sauce...",8.0
26170,2001102678,530285,2017-05-01,5,a fun fruitful way to start the day,fruity pebbles parfait,7,33186,2017-02-08,"['15-minutes-or-less', 'time-to-make', 'course...","[178.8, 1.0, 101.0, 0.0, 5.0, 0.0, 14.0]",6,['place a small layer of the yogurt in the bot...,indulge in a colorful treat while still gettin...,"['greek yogurt', 'fruity pebbles cereal', 'ban...",6.0
10295,45027729,77039,2017-09-13,1,"I also say WHY?, but mine is &quot;why make it...",microwave cabbage soup,45,27783,2003-11-20,"['60-minutes-or-less', 'time-to-make', 'course...","[210.4, 16.0, 33.0, 58.0, 20.0, 30.0, 6.0]",9,"['you will need a deep , heavy microwave safe ...",i ate a cabbage soup at a restaurant that was ...,"['green cabbage', 'unsalted butter', 'onions',...",10.0
20163,1274599,72197,2016-02-08,5,This came out so good. I followed the directi...,white pie,50,95743,2003-10-01,"['60-minutes-or-less', 'time-to-make', 'course...","[421.2, 29.0, 145.0, 12.0, 14.0, 46.0, 18.0]",6,"['in a saucepan cook flour , cornstarch , suga...",in southern il burton's cafe is famous for the...,"['flour', 'cornstarch', 'sugar', 'salt', 'egg ...",12.0
24630,2000786197,424162,2017-05-17,0,It's not Egyptian if it doesn't have chickpeas...,egyptian koshari,75,171790,2010-05-09,"['weeknight', 'time-to-make', 'course', 'main-...","[436.4, 12.0, 8.0, 19.0, 26.0, 5.0, 25.0]",21,"['for lentils and rice , heat oil in large pot...",shadya saleh of lotus catering & fine foods (1...,"['vegetable oil', 'yellow onion', 'salt', 'gro...",15.0


# Feature Engineering

## Data Shape Details

In [25]:
# Viewing test & train dataset shape

train.shape, test.shape

((13654, 16), (5852, 16))

## Data Information Details

In [26]:
# Checking train data information

train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13654 entries, 33780 to 64428
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_id         13654 non-null  int64         
 1   recipe_id       13654 non-null  int64         
 2   date            13654 non-null  datetime64[ns]
 3   rating          13654 non-null  int64         
 4   review          13641 non-null  object        
 5   name            13654 non-null  object        
 6   minutes         13654 non-null  int64         
 7   contributor_id  13652 non-null  object        
 8   submitted       13654 non-null  datetime64[ns]
 9   tags            13654 non-null  object        
 10  nutrition       13652 non-null  object        
 11  n_steps         13654 non-null  int64         
 12  steps           13654 non-null  object        
 13  description     13654 non-null  object        
 14  ingredients     13654 non-null  object        
 15  n_i

In [27]:
# Checking test data information

test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5852 entries, 12224 to 30432
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_id         5852 non-null   int64         
 1   recipe_id       5852 non-null   int64         
 2   date            5852 non-null   datetime64[ns]
 3   rating          5852 non-null   int64         
 4   review          5850 non-null   object        
 5   name            5852 non-null   object        
 6   minutes         5852 non-null   int64         
 7   contributor_id  5851 non-null   object        
 8   submitted       5852 non-null   datetime64[ns]
 9   tags            5852 non-null   object        
 10  nutrition       5851 non-null   object        
 11  n_steps         5852 non-null   int64         
 12  steps           5852 non-null   object        
 13  description     5852 non-null   object        
 14  ingredients     5852 non-null   object        
 15  n_in

# Unused feature Removal

In [28]:
# Defining custom function to remove unused features in dataframe

def unused_feat_removal(df, feature_to_remove):
    df.drop(columns=feature_to_remove, inplace=True)
    return df

In [29]:
# initiatlizing unused column in dataset to be removed

feature_to_remove = ["date", "review", "name", "minutes", "contributor_id", "submitted",
                     "tags", "nutrition", "n_steps", "steps", "description", "ingredients", "n_ingredients"]

In [30]:
# removing used columns in train & test datasets

unused_feat_removal(train, feature_to_remove)
unused_feat_removal(test, feature_to_remove)

# Viewing test & traing dataset shape after column removal
train.shape, test.shape

((13654, 3), (5852, 3))

In [31]:
# Viewing traing dataset

train.head()

,user_id,recipe_id,rating
33780,794029,139518,4
44565,128473,536070,5
56295,2001345166,20376,5
61487,2001276950,293275,5
16305,1741243,257554,2


In [32]:
# Viewing test dataset

test.head()

,user_id,recipe_id,rating
12224,2000297339,177369,0
26170,2001102678,530285,5
10295,45027729,77039,1
20163,1274599,72197,5
24630,2000786197,424162,0


# User-item Matrix

In [33]:
def create_mappings(train, test):
    user_mapping = {user: i for i, user in enumerate(
        train["user_id"].unique())}
    all_recipe_ids = np.union1d(
        train["recipe_id"].unique(), test["recipe_id"].unique())
    recipe_mapping = {recipe: i for i, recipe in enumerate(all_recipe_ids)}
    return user_mapping, recipe_mapping


def apply_mappings(df, user_mapping, recipe_mapping):
    df["user_id_mapped"] = df["user_id"].map(user_mapping).astype(int)
    df["recipe_id_mapped"] = df["recipe_id"].map(recipe_mapping).astype(int)
    return df


def create_sparse_user_item_matrix(df, n_users, n_items):
    rows = df["user_id_mapped"].values
    cols = df["recipe_id_mapped"].values
    data = df["rating"].values
    return csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

In [34]:
user_mapping, recipe_mapping = create_mappings(train, test)
train = apply_mappings(train, user_mapping, recipe_mapping)
test = apply_mappings(test, user_mapping, recipe_mapping)
n_users = len(np.unique(np.concatenate(
    [train["user_id_mapped"].values, test["user_id_mapped"].values])))
n_items = len(np.unique(np.concatenate(
    [train["recipe_id_mapped"].values, test["recipe_id_mapped"].values])))
train_sparse_matrix = create_sparse_user_item_matrix(train, n_users, n_items)
test_sparse_matrix = create_sparse_user_item_matrix(test, n_users, n_items)

# Feature Inspection

In [35]:
train_sparse_matrix.shape, test_sparse_matrix.shape

((4448, 12396), (4448, 12396))

In [36]:
def calculate_sparsity(sparse_matrix):
    matrix_size = sparse_matrix.shape[0] * sparse_matrix.shape[1]
    num_non_zero = sparse_matrix.nnz
    sparsity = (1 - (num_non_zero / matrix_size)) * 100
    return sparsity

In [37]:
train_sparsity = calculate_sparsity(train_sparse_matrix)
test_sparsity = calculate_sparsity(test_sparse_matrix)

print(f"Training Matrix Sparsity: {train_sparsity}%")
print(f"Testing Matrix Sparsity: {test_sparsity}%")

Training Matrix Sparsity: 99.97523822665005%
Testing Matrix Sparsity: 99.98938651595665%


In [38]:
train_dense_submatrix = train_sparse_matrix[:10, :10].todense()
train_submatrix = pd.DataFrame(train_dense_submatrix)
train_submatrix

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,5,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


In [39]:
test_dense_submatrix = train_sparse_matrix[:10, :10].todense()
test_submatrix = pd.DataFrame(test_dense_submatrix)
test_submatrix

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,5,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


# Exporting Data

In [40]:
# Exporting datas

with open('../../data/processed/knn-model/train_sparse_matrix.pkl', 'wb') as file:
    pickle.dump(train_sparse_matrix, file)

with open('../../data/processed/knn-model/test_sparse_matrix.pkl', 'wb') as file:
    pickle.dump(test_sparse_matrix, file)

with open('../../data/processed/knn-model/user_mapping.pkl', 'wb') as file:
    pickle.dump(user_mapping, file)

with open('../../data/processed/knn-model/recipe_mapping.pkl', 'wb') as file:
    pickle.dump(recipe_mapping, file)